In [11]:
import module_luis.get_data as get_data
import module_luis.luis_function as lf
import module_luis.luis_prediction as luis_predict

<div class="alert alert-block alert-info">
<h2><center><strong>Creation des données pour Luis </strong></center></h2>
        
</div>

In [3]:
%load_ext pycodestyle_magic

In [4]:
%flake8_on

<div class="alert alert-block alert-success">  
<strong>1. Chargement du jeu de données<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

*a) Initialisation des paramètres*

In [12]:
data_path = "data/frames.json"

*b) Chargement du fichier*

In [13]:
data_list = get_data.load_data(data_path)

On le transforme pour qu'il soit utilisable dans luis :

In [39]:
list_final = get_data.transform_data(data_list)
list_train = list_final[99:]
list_test = list_final[0:99]

### Creation de l'application luis, des intentions et des utterances :

In [30]:
print("Creating application...")
app_id, app_version = lf.create_app()
print(app_id, app_version)

Creating application...
Created LUIS app Fly Bot
    with ID 9a4d53de-3156-4e7a-80ec-30803825ce2f
9a4d53de-3156-4e7a-80ec-30803825ce2f 0.1


In [31]:
print("Adding entities to application...")
lf.add_entities(app_id, app_version)

Adding entities to application...
destinationEntityId a4b5d7d5-c8be-4b34-8738-f1855aa82f12 added.
classEntityId 429dd0f9-ecba-4626-84f6-8a3bc803d6bf added.
flightEntityId 2b23f6b1-739b-4687-823c-013850bdb4ad added.
flightEntityId eb5c2cff-7834-4215-a7b8-47c8f00619e7 added.
flightEntityId a4910fa0-e380-4527-a007-7e896b7d3a1b added.


In [32]:
print("Adding intents to application...")
lf.add_intents(app_id, app_version)

Adding intents to application...
Intent BookFlight 9c0bef58-6c19-4da7-b525-ba5a0aaa9c13 added.
Intent Cancel b2d81b0e-cfba-463d-b6ad-3abdde5f7313 added.
Intent Greeting d4f3d899-6601-4d53-bffe-fe382a035c26 added.
Intent Help d1fb3781-8d1e-465a-a7ba-ae3a8dd942ce added.
Intent Closing dd9c2e8f-15f5-43ef-8052-7cdde5661917 added.


on ajoute les entités correspondant à Greeting, Cancel, Help, None...

In [33]:
intentions_path = "data/data_intentions.txt"
list_utterances_default = get_data.create_data(intentions_path)
lf.add_utterances(app_id, app_version, list_utterances_default)
print("{} example utterance(s) added.".format(len(list_utterances_default)))

45 example utterance(s) added.


On les ajoute 100 par 100 sinon Luis refuse autant de requetes en gratuit : 

In [34]:
print("Adding intents to utterances...")
# on garde les 229 premieres utterances pour evaluer luis
for i in range(229, len(list_train), 100):
    j = (i + 100)
    if j > len(list_final):
        j = len(list_final)
    lf.add_utterances(app_id, app_version, list_train[i:j])
print("{} example utterance(s) added.".format(len(list_train)))

Adding intents to utterances...
5129 example utterance(s) added.


In [35]:
print("Train application...")
lf.train_app(app_id, app_version)

Train application...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [42]:
print("Evaluate application...")
test_path = "data/evaluate.json"
lf.evaluate_app(test_path, list_test)

Evaluate application...


In [37]:
print("Publish application...")
lf.publish_app(app_id, app_version)

Publish application...
Application published. Endpoint URL: https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/9a4d53de-3156-4e7a-80ec-30803825ce2f


<div class="alert alert-block alert-success">  
<strong>2. Evaluation des predictions<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [11]:
# <OtherVariables>
# Use public app ID or replace with your own trained and published app's ID
# to query your own app
# public appID = 'df67dcdb-c37d-46af-88e1-8b97951ca1c2'
luisAppID = '5705e735-228d-430c-94fd-cab67b7f69ae'
print("luisAppID: {}".format(luisAppID))

# `production` or `staging`
luisSlotName = 'production'
print("luisSlotName: {}".format(luisSlotName))
# </OtherVariables>


luisAppID: 5705e735-228d-430c-94fd-cab67b7f69ae
luisSlotName: production


In [13]:
luis_predict.predict(luisAppID, luisSlotName)

Top intent: BookFlight
Sentiment: None
Intents: 
	"BookFlight"
Entities: {'or_city': ['paris'], 'dst_city': ['berlin'], 'budget': ['2020'], 'str_date': ['september'], 'end_date': ['2nd october']}
